<a href="https://colab.research.google.com/github/liamti5/UZH-Essentials-in-Text-and-Speech-Processing/blob/master/code/LiamModel4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!pip install transformers[torch]

In [28]:
# !pip install transformers

In [29]:
!pip install datasets

In [30]:
from datasets import load_dataset

imdb = load_dataset('/content/drive/My Drive/Colab/Uni/HS23/EITASP/', data_files={'train': 'train.csv', 'test': 'test.csv', 'validation': 'validation.csv'})
# tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [31]:
imdb['train'][0]

{'Unnamed: 0': 2347,
 '_id': '61e0642e1875bb0021077478',
 'review': 'Alles irreführend mit diesem Kurs: Prüfungsrelevante Inhalte, Prüfungsfragen, Aufsicht oder keine Aufsicht, Dias und vieles mehr!',
 'score': 1,
 'upvotes': 0.0,
 'downvotes': 0.0,
 'sum': 0}

In [32]:
imdb = imdb.rename_column("review", "text")
imdb = imdb.rename_column("sum", "label")

In [33]:
imdb = imdb.remove_columns(["Unnamed: 0", "_id", "score", "upvotes", "downvotes"])

In [34]:
imdb['train'][1]

{'text': 'leider ist die prüfung praktisch nur glückssache... schlechtes modul mit herzloser dozentin! ',
 'label': 3}

In [35]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-german-cased")

In [36]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [37]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [38]:
tokenized_imdb['train'][0]

{'text': 'Alles irreführend mit diesem Kurs: Prüfungsrelevante Inhalte, Prüfungsfragen, Aufsicht oder keine Aufsicht, Dias und vieles mehr!',
 'label': 0,
 'input_ids': [102,
  4246,
  20020,
  5658,
  30888,
  212,
  893,
  6490,
  853,
  17639,
  8311,
  12033,
  3882,
  11154,
  818,
  17639,
  4789,
  818,
  15641,
  394,
  855,
  15641,
  818,
  14903,
  30886,
  136,
  13200,
  484,
  3330,
  103],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [39]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [40]:
!pip install evaluate

In [41]:
import evaluate

accuracy = evaluate.load("accuracy")

In [42]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [43]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-german-cased", num_labels=4
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
import torch

In [45]:
# Check if GPU is available
if torch.cuda.is_available():
    print("using GPU")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

using GPU


In [46]:
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/Colab/Uni/HS23/EITASP/trial2",
    learning_rate=2e-5,  # Adjusted learning rate
    per_device_train_batch_size=16,  # Smaller train batch size
    per_device_eval_batch_size=16,  # Smaller eval batch size
    num_train_epochs=4,  # More epochs
    weight_decay=0.01,  # Added weight decay for regularization
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    gradient_accumulation_steps=2,  # Added gradient accumulation
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.885889,0.657063
2,0.898300,0.885824,0.639494
3,0.898300,0.893468,0.635278


TrainOutput(global_step=956, training_loss=0.8487317711738362, metrics={'train_runtime': 431.2299, 'train_samples_per_second': 71.015, 'train_steps_per_second': 2.217, 'total_flos': 1107390366110784.0, 'train_loss': 0.8487317711738362, 'epoch': 3.99})

In [22]:
trainer.save_model("/content/drive/My Drive/Colab/Uni/HS23/EITASP/model")

In [23]:
model = DistilBertForSequenceClassification.from_pretrained(
    "/content/drive/My Drive/Colab/Uni/HS23/EITASP/model", num_labels=4
)